# Traffic Light Frozen Inference Model Tests

### Real Track Test Images

Tests of traffic light detection/localization/classification on Udacity real track images. 

# Imports

In [1]:
import numpy as np
import os
import sys
import tensorflow as tf

from collections import defaultdict
from io import StringIO
from matplotlib import pyplot as plt
from PIL import Image

import time

# Tested with tensorflow 1.3


In [2]:
# This is needed to display the images withing the notbook.
%matplotlib inline

## Object detection imports
Here are the imports from the object detection module.

In [3]:
from utils import label_map_util

from utils import visualization_utils as vis_util

## Helper code

In [4]:
def load_image_into_numpy_array(image):
  (im_width, im_height) = image.size
  return np.array(image.getdata()).reshape(
      (im_height, im_width, 3)).astype(np.uint8)

## Loading label map
Label maps map indices to category names, so that when our convolution network predicts `1`, we know that this corresponds to `Green`, 2 to `Red`, and 3 to `Yellow`.

In [5]:
# List of the strings that is used to add correct label for each box.
PATH_TO_LABELS = 'tl_label_map.pbtxt'

NUM_CLASSES = 3

label_map = label_map_util.load_labelmap(PATH_TO_LABELS)
categories = label_map_util.convert_label_map_to_categories(label_map, max_num_classes=NUM_CLASSES, use_display_name=True)
category_index = label_map_util.create_category_index(categories)

# Model preparation for trial19_ssd_inception_sim_frozen_inference_graph

In [10]:
# What model to download.

# Path to frozen detection graph. This is the actual model that is used for the object detection.
PATH_TO_CKPT = os.path.join('models', 'trial19_ssd_inception_sim_frozen_inference_graph.pb')

### Load a (frozen) Tensorflow model into memory.

In [11]:
detection_graph = tf.Graph()
with detection_graph.as_default():
  od_graph_def = tf.GraphDef()
  with tf.gfile.GFile(PATH_TO_CKPT, 'rb') as fid:
    serialized_graph = fid.read()
    od_graph_def.ParseFromString(serialized_graph)
    tf.import_graph_def(od_graph_def, name='')
  
    sess = tf.Session(graph=detection_graph)
    
# Definite input and output Tensors for detection_graph
image_tensor = detection_graph.get_tensor_by_name('image_tensor:0')
# Each box represents a part of the image where a particular object was detected.
detection_boxes = detection_graph.get_tensor_by_name('detection_boxes:0')
# Each score represent how level of confidence for each of the objects.
# Score is shown on the result image, together with the class label.
detection_scores = detection_graph.get_tensor_by_name('detection_scores:0')
detection_classes = detection_graph.get_tensor_by_name('detection_classes:0')
num_detections = detection_graph.get_tensor_by_name('num_detections:0')

## trial19_ssd_inception_sim_frozen_inference_graph
## Detection times on Real Images from Test Track without warmup image

In [12]:
from glob import glob
# If you want to test the code with your images, just add path to the images to the TEST_IMAGE_PATHS.
PATH_TO_TEST_IMAGES_DIR = 'test_images_real'
TEST_IMAGE_PATHS = glob(os.path.join(PATH_TO_TEST_IMAGES_DIR, '*.jpg'))

# Size, in inches, of the output images.
IMAGE_SIZE = (12, 8)

In [13]:
# The time for the first image includes startup time for tensorflow
with detection_graph.as_default():
    for image_path in TEST_IMAGE_PATHS:
        image = Image.open(image_path)
        # the array based representation of the image will be used later in order to prepare the
        # result image with boxes and labels on it.
        image_np = load_image_into_numpy_array(image)
        # Expand dimensions since the model expects images to have shape: [1, None, None, 3]
        image_np_expanded = np.expand_dims(image_np, axis=0)

        # Actual detection.
        time0 = time.time()
        (boxes, scores, classes, num) = sess.run(
            [detection_boxes, detection_scores, detection_classes, num_detections],
            feed_dict={image_tensor: image_np_expanded})
        time1 = time.time()

        # Visualization of the results of a detection.
    #     vis_util.visualize_boxes_and_labels_on_image_array(
    #         image_np,
    #         np.squeeze(boxes),
    #         np.squeeze(classes).astype(np.int32),
    #         np.squeeze(scores),
    #         category_index,
    #         use_normalized_coordinates=True,
    #         line_thickness=7)
    #     plt.figure(figsize=IMAGE_SIZE)
    #     plt.rcParams.update({'figure.max_open_warning': 0})
    #     plt.imshow(image_np)
    #     plt.show()
        print("Time in ms: ", (time1 - time0) * 1000, "\n")

Time in ms:  1437.323808670044 

Time in ms:  16.04461669921875 

Time in ms:  16.54529571533203 

Time in ms:  18.049240112304688 

Time in ms:  17.548322677612305 

Time in ms:  16.545534133911133 

Time in ms:  17.04716682434082 

Time in ms:  15.542745590209961 

Time in ms:  17.548084259033203 

Time in ms:  17.046689987182617 

Time in ms:  17.5478458404541 

Time in ms:  17.548084259033203 

Time in ms:  16.545534133911133 

Time in ms:  17.04692840576172 

Time in ms:  17.046689987182617 

Time in ms:  17.04573631286621 

Time in ms:  16.54338836669922 

Time in ms:  16.044139862060547 

Time in ms:  16.042470932006836 

Time in ms:  17.045259475708008 

Time in ms:  17.546653747558594 

Time in ms:  16.043901443481445 

Time in ms:  17.046451568603516 

Time in ms:  16.54505729675293 

Time in ms:  17.04692840576172 

Time in ms:  16.545534133911133 

Time in ms:  16.042232513427734 

Time in ms:  16.543865203857422 

Time in ms:  18.047332763671875 

Time in ms:  16.042470932

# Model preparation for trial19_ssd_inception_sim_frozen_inference_graph

In [6]:
# What model to download.

# Path to frozen detection graph. This is the actual model that is used for the object detection.
PATH_TO_CKPT = os.path.join('models', 'trial19_ssd_inception_sim_frozen_inference_graph.pb')

### Load a (frozen) Tensorflow model into memory and run an image through to get it going.

In [7]:
detection_graph = tf.Graph()
with detection_graph.as_default():
  od_graph_def = tf.GraphDef()
  with tf.gfile.GFile(PATH_TO_CKPT, 'rb') as fid:
    serialized_graph = fid.read()
    od_graph_def.ParseFromString(serialized_graph)
    tf.import_graph_def(od_graph_def, name='')
  
    sess = tf.Session(graph=detection_graph)
    
# Definite input and output Tensors for detection_graph
image_tensor = detection_graph.get_tensor_by_name('image_tensor:0')
# Each box represents a part of the image where a particular object was detected.
detection_boxes = detection_graph.get_tensor_by_name('detection_boxes:0')
# Each score represent how level of confidence for each of the objects.
# Score is shown on the result image, together with the class label.
detection_scores = detection_graph.get_tensor_by_name('detection_scores:0')
detection_classes = detection_graph.get_tensor_by_name('detection_classes:0')
num_detections = detection_graph.get_tensor_by_name('num_detections:0')

image = Image.open('test_images_real/left0314.jpg')
# the array based representation of the image will be used later in order to prepare the
# result image with boxes and labels on it.
image_np = load_image_into_numpy_array(image)
# Expand dimensions since the model expects images to have shape: [1, None, None, 3]
image_np_expanded = np.expand_dims(image_np, axis=0)

(boxes, scores, classes, num) = sess.run(
            [detection_boxes, detection_scores, detection_classes, num_detections],
            feed_dict={image_tensor: image_np_expanded})

## trial19_ssd_inception_sim_frozen_inference_graph
## Detection times on Real Images from Test Track with warmup image.

In [8]:
from glob import glob
# If you want to test the code with your images, just add path to the images to the TEST_IMAGE_PATHS.
PATH_TO_TEST_IMAGES_DIR = 'test_images_real'
TEST_IMAGE_PATHS = glob(os.path.join(PATH_TO_TEST_IMAGES_DIR, '*.jpg'))

# Size, in inches, of the output images.
IMAGE_SIZE = (12, 8)

In [9]:
# The time for the first image includes startup time for tensorflow
with detection_graph.as_default():
    for image_path in TEST_IMAGE_PATHS:
        image = Image.open(image_path)
        # the array based representation of the image will be used later in order to prepare the
        # result image with boxes and labels on it.
        image_np = load_image_into_numpy_array(image)
        # Expand dimensions since the model expects images to have shape: [1, None, None, 3]
        image_np_expanded = np.expand_dims(image_np, axis=0)

        # Actual detection.
        time0 = time.time()
        (boxes, scores, classes, num) = sess.run(
            [detection_boxes, detection_scores, detection_classes, num_detections],
            feed_dict={image_tensor: image_np_expanded})
        time1 = time.time()

        # Visualization of the results of a detection.
    #     vis_util.visualize_boxes_and_labels_on_image_array(
    #         image_np,
    #         np.squeeze(boxes),
    #         np.squeeze(classes).astype(np.int32),
    #         np.squeeze(scores),
    #         category_index,
    #         use_normalized_coordinates=True,
    #         line_thickness=7)
    #     plt.figure(figsize=IMAGE_SIZE)
    #     plt.rcParams.update({'figure.max_open_warning': 0})
    #     plt.imshow(image_np)
    #     plt.show()
        print("Time in ms: ", (time1 - time0) * 1000, "\n")

Time in ms:  16.042709350585938 

Time in ms:  16.54338836669922 

Time in ms:  16.544103622436523 

Time in ms:  17.548322677612305 

Time in ms:  17.04692840576172 

Time in ms:  15.542984008789062 

Time in ms:  16.54505729675293 

Time in ms:  17.046689987182617 

Time in ms:  17.548322677612305 

Time in ms:  17.046689987182617 

Time in ms:  17.047405242919922 

Time in ms:  16.54505729675293 

Time in ms:  17.046689987182617 

Time in ms:  18.04971694946289 

Time in ms:  16.544103622436523 

Time in ms:  16.544342041015625 

Time in ms:  16.043901443481445 

Time in ms:  16.544818878173828 

Time in ms:  17.04573631286621 

Time in ms:  17.046213150024414 

Time in ms:  16.54529571533203 

Time in ms:  17.04716682434082 

Time in ms:  17.045021057128906 

Time in ms:  17.548799514770508 

Time in ms:  16.04485511779785 

Time in ms:  17.54927635192871 

Time in ms:  17.045259475708008 

Time in ms:  17.547607421875 

Time in ms:  15.542268753051758 

Time in ms:  16.04437828063